In [3]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size']=16

In [4]:
from laplace_helper import p_analytical, plot_3D, L2_rel_error

In [5]:
ny = 128
nx = 128

L = 5
H = 5

x = numpy.linspace(0, L, nx)
y = numpy.linspace(0, H, ny)
dx = L/(nx-1)
dy = H/(nx-1)

p0 = numpy.zeros((ny, nx))
p0[-1,:] = numpy.sin(1.5*numpy.pi*x/x[-1])

In [6]:
def laplace2d(p, l2_target):
    '''Solves the Laplace eqn using the Jacobi method with a 5-point stencil
    
    Parameters:
        p - initial potential distribution
        l2_target - stopping criteria
    Returns:
        p - potential distribution after relaxation
    '''
    
    l2norm = 1
    pn = numpy.empty_like(p)
    iterations = 0
    while l2norm > l2_target:
        pn = p.copy()
        p[1:-1, 1:-1] = .25*(pn[1:-1, 2:] + pn[1:-1, :-2] +\
                            pn[2:, 1:-1] + pn[:-2, 1:-1])
        
        # Neumann BCs at x=L
        p[1:-1, -1] = .25*(2*pn[1:-1, -2] + pn[2:, -1] + pn[:-2, -1])
        
        l2norm = numpy.sqrt(numpy.sum(p-pn)**2/numpy.sum(pn**2))
        iterations +=1
    
    return p, iterations

In [7]:
l2_target = 1e-8
p, iterations = laplace2d(p0.copy(), l2_target)

print("Jacobi method too k {} iterations at {} tolerance".format(iterations, l2_target))

Jacobi method too k 27057 iterations at 1e-08 tolerance


In [6]:
%%timeit
laplace2d(p0.copy(), l2_target)

1 loops, best of 3: 4.87 s per loop


In [8]:
pan = p_analytical(x,y)

In [9]:
L2_rel_error(p, pan)

7.7434413870791999e-05

In [10]:
import numba 
from numba import jit

In [11]:
@jit
def fib_it(n):
    a = 1
    b = 1
    for i in range(n-2):
        a, b = b, a+b
        
    return b

In [12]:
%%timeit
fib_it(500000)

The slowest run took 159.26 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 399 µs per loop


In [13]:
@jit(nopython=True)
def laplace2d_jacobi(p, pn, l2_target):
    '''Solves the Laplace equation using the Jacobi method with a 5-point stencil
    
    Parameters:
        p - initial potential distribution
        pn - allocated array for previous potential distribution
        l2_target - stopping criteria
        
    Returns:
        p - potential distribution after relaxation
    '''
    
    iterations = 0
    iter_diff = l2_target+1
    denominator = 0.0
    ny, nx = p.shape
    l2_diff = numpy.zeros(20000)
    
    while iter_diff > l2_target:
        for j in range(ny):
            for i in range(nx):
                pn[j, i] = p[j,i]
                
        iter_diff = 0.0
        denominator = 0.0
        
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                p[j,i] = .25*(pn[j,i-1] + pn[j,i+1] + pn[j-1,i] + pn[j+1, i])
                
        for j in range(ny):
            for i in range(nx):
                iter_diff += (p[j,i] - pn[j,i])**2
                denominator += (pn[j,i]*pn[j,i])
                
        iter_diff /= denominator
        iter_diff = iter_diff**0.5
        l2_diff[iterations] = iter_diff
        iterations +=1
        
    return p, iterations, l2_diff
        

In [13]:
p, iterations, l2_diffJ = laplace2d_jacobi(p0.copy(), p0.copy(), 1e-8)
print("Numba Jacobi method took {} iterations at tolerance {}".format(iterations, l2_target))

Numba Jacobi method took 30540 iterations at tolerance 1e-08


In [ ]:
## NOTE: this code kills the jupyter kernel on desktop.
%%timeit
laplace2d_jacobi(p0.copy(), p0.copy(), 1e-8)

In [14]:
@jit(nopython=True)
def laplace2d_gauss_seidel(p, pn, l2_target):
    '''Solves the Laplace equation using Gauss-Seidel method with 5-point stencil
    
    Parameters:
        p - initial potential distribution
        pn - allocated array for previous potential distribution
        l2_target - stopping criteria
    
    Returns:
        p - potential distribution after relaxation
    '''
    
    iterations = 0 
    iter_diff = l2_target +1
    denominator = 0.0
    ny, nx = p.shape
    l2_diff = numpy.zeros(20000)
    
    while iter_diff > l2_target:
        for j in range(ny):
            for i in range(nx):
                p[j,i] = p[j,i]
        iter_diff = 0.0
        denominator = 0.0
        
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                p[j,i] = .25*(p[j, i-1]+p[j,i+1] + p[j-1,i] + p[j+1,i])
                
        ## BCs
        for j in range(1, ny-1):
            p[j,-1] = .25*(2*p[j,-2] + p[j+1, -1] + p[j-1,-1])
                
        for j in range(ny):
            for i in range(nx):
                iter_diff += (p[j,i] - pn[j,i])**2
                denominator += (pn[j,i]*pn[j,i])
                
        iter_diff /= denominator
        iter_diff = iter_diff**0.5
        l2_diff[iterations] = iter_diff
        iterations +=1
    
    return p, iterations, l2_diff
            

In [ ]:
p, iterations, l2_diffGS = laplace2d_gauss_seidel(p0.copy(), p0.copy(), 1e-8)

print("Numba Gauss-Seidel method took {} iterations at tolerance {}".format(iterations, l2_target))